### 1. Importy + ścieżki

In [13]:
### 7. Importy i konfiguracja ścieżek

from pathlib import Path
import pandas as pd
import re
from IPython.display import display
from ipywidgets import widgets, VBox, HBox, Output
from PIL import Image

PROJECT_ROOT = Path(".").resolve()
ROOT = PROJECT_ROOT  # alias, żeby dalsze komórki mogły używać ROOT

OUTPUT_CSV_DIR = PROJECT_ROOT / "outputs" / "csv"
OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)

# Pomocnicza funkcja do rozbijania tekstów typu "person; tie"
def clean_split(value: str):
    if not isinstance(value, str):
        return []
    tmp = value.replace("(", ";").replace(")", ";")
    parts = [p.strip() for p in re.split(r"[;,]", tmp)]
    return [p for p in parts if p]

### 2. CLIP → df_clip

In [6]:
### 7.0 CLIP – wczytanie wyników z CSV

clip_csv = OUTPUT_CSV_DIR / "clip_scene_subjects.csv"

if not clip_csv.exists():
    print("[07] Brak pliku:", clip_csv)
    df_clip = pd.DataFrame(columns=["file_name", "subject_en", "subject_pl"])
else:
    df_raw = pd.read_csv(clip_csv)
    print("[07] CLIP kolumny:", list(df_raw.columns))

    needed = {"file_name", "subject_en", "subject_pl"}
    missing = needed - set(df_raw.columns)
    if missing:
        raise ValueError(f"Brak kolumn w {clip_csv}: {missing}")

    # bierzemy tylko to, co potrzebne do kandydatów
    df_clip = df_raw[["file_name", "subject_en", "subject_pl"]].copy()
    print("[07] df_clip shape:", df_clip.shape)

df_clip.head()

[07] CLIP kolumny: ['file_name', 'subject_en', 'subject_pl', 'scene_score']
[07] df_clip shape: (74, 3)


,file_name,subject_en,subject_pl
0,0004.jpg,shop interior,wnętrze sklepu
1,0006.jpg,crowd of people on the street,tłum na ulicy
2,0009.jpg,graffiti or political slogans on walls,napisy lub hasła na murach
3,0022.jpg,street protest,demonstracja uliczna
4,0034.jpg,strollers or playgrounds,wózki dziecięce lub place zabaw


### 4. OCR → df_ocr (wersja prosta)

In [7]:
### 7.2 OCR – pozyskanie słów jako subjectów (wersja prosta)

ocr_csv = OUTPUT_CSV_DIR / "ocr_results.csv"

if ocr_csv.exists():
    df_ocr_raw = pd.read_csv(ocr_csv)
else:
    df_ocr_raw = pd.DataFrame(columns=["file_path","full_text"])

def extract_words(text):
    if not isinstance(text,str):
        return []
    # słowa ≥ 3 litery
    return re.findall(r"[A-Za-zĄąĆćĘęŁłŃńÓóŚśŻżŹź]{3,}", text)

rows = []
for _, row in df_ocr_raw.iterrows():
    iid = row["file_path"]
    text = row.get("full_text","")
    for w in extract_words(text):
        rows.append({
            "image_id": iid,
            "subject_label_pl": w.lower(),
            "subject_label_en": w.lower(),
            "source": "ocr",
        })

df_ocr = pd.DataFrame(rows).drop_duplicates()
df_ocr.head()

KeyError: 'file_path'

### 5. Ścieżki plików i dane wejściowe

In [16]:
"""
    Parametry przeglądu i pliki stanu.

    - PREVIEW: podglądy z ramek LVIS (jeśli istnieją), inaczej obraz źródłowy.
    - STATE: zapis zatwierdzonych subjects per file_name (wznawialny).
"""

from pathlib import Path
import pandas as pd

ROOT = Path("/Users/olga/MetaLogic")
P_INPUTS = ROOT / "inputs"
P_PREV   = ROOT / "outputs/lvis_try/preview"  # miniatury z LVIS
P_LVIS   = ROOT / "outputs/csv/lvis_subjects_en_pl.csv"
P_OCR    = ROOT / "outputs/csv/ocr_results.csv"  # opcjonalnie do dc.description

P_REVIEW = ROOT / "outputs/review/review_subjects.csv"
P_REVIEW.parent.mkdir(parents=True, exist_ok=True)

def read_csv_safe(p: Path) -> pd.DataFrame:
    return pd.read_csv(p) if p.exists() else pd.DataFrame()

df_lvis = read_csv_safe(P_LVIS)
df_ocr  = read_csv_safe(P_OCR)

if P_REVIEW.exists():
    df_review = pd.read_csv(P_REVIEW)
else:
    df_review = pd.DataFrame(
        columns=["file_name", "subjects_pl", "subjects_en", "notes", "reviewed"]
    )

inputs_all = sorted([
    p.name for p in P_INPUTS.iterdir()
    if p.suffix.lower() in {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
])

### 6. Proponowane subjects

In [9]:
"""
Kandydaci subjects per-obraz:
- LVIS: unikalne etykiety EN/PL dla danego pliku (max topk).
- CLIP: sceny EN/PL (jeśli są).
- Słownik globalny PL (VOCAB_PL) dodawany tylko, gdy włączysz przełącznik w UI.
"""

LVIS_TOP_DEFAULT = 12

def _split_semis(s: str):
    return [t.strip() for t in str(s or "").split(";") if t and t.strip()]

def _vocab_pl_from_data():
    """
    Buduje słownik PL z dotychczasowych recenzji (df_review).
    Służy jako dodatkowa lista propozycji, gdy w UI zaznaczysz add_vocab_pl.
    """
    pool = []
    if not df_review.empty and "subjects_pl" in df_review.columns:
        for s in df_review["subjects_pl"].dropna().astype(str):
            pool += _split_semis(s)

    seen, out = set(), []
    for w in pool:
        k = w.lower()
        if k not in seen:
            seen.add(k)
            out.append(w)
    return out

VOCAB_PL = _vocab_pl_from_data()

def candidates_for(fname: str, topk: int = LVIS_TOP_DEFAULT, add_vocab_pl: bool = False):
    cand_pl, cand_en = [], []

    # LVIS – korzystamy z pliku lvis_subjects_en_pl.csv
    if not df_lvis.empty:
        d = df_lvis[df_lvis["file_name"] == fname]
        if not d.empty:
            en_list = (
                d["subject_en"]
                .dropna()
                .astype(str)
                .tolist()
            )
            pl_list = (
                d["subject_pl"]
                .dropna()
                .astype(str)
                .tolist()
            )

            # ograniczenie do topk unikalnych
            def _unique(seq):
                seen, out = set(), []
                for x in seq:
                    k = x.lower()
                    if k not in seen:
                        seen.add(k)
                        out.append(x)
                return out

            en_list = _unique(en_list)[: int(topk)]
            pl_list = _unique(pl_list)[: int(topk)]

            cand_en += en_list
            cand_pl += pl_list

    # CLIP – sceny EN/PL (jeśli df_clip istnieje)
    if "df_clip" in globals() and df_clip is not None and not df_clip.empty:
        d = df_clip[df_clip["file_name"] == fname]
        if not d.empty:
            cand_en += [
                str(x) for x in d["subject_en"].dropna().astype(str).tolist()
            ]
            cand_pl += [
                str(x) for x in d["subject_pl"].dropna().astype(str).tolist()
            ]

    # dodatkowy słownik PL (z recenzji) – opcjonalnie
    if add_vocab_pl:
        cand_pl += VOCAB_PL

    # deduplikacja
    def dedup(seq):
        seen, out = set(), []
        for x in [s for s in seq if s]:
            k = x.lower()
            if k not in seen:
                seen.add(k)
                out.append(x)
        return out

    return dedup(cand_pl), dedup(cand_en)

### 7. Normalizacja klucza i porządkowanie kolumn

In [10]:
### 7. Normalizacja klucza i porządkowanie kolumn (LVIS/CLIP/OCR)

from pathlib import Path
import pandas as pd

def _normalize_subject_df(df: pd.DataFrame | None) -> pd.DataFrame | None:
    """
    Ujednolica format ramek z propozycjami subjectów.

    Docelowo:
    - kolumna 'file_name' (tylko nazwa pliku),
    - kolumny 'subject_en' i 'subject_pl' (jeśli brak – pusty string),
    - 'file_name' jako pierwsza kolumna.

    Jeśli ramka nie ma kolumny identyfikującej plik (file_name/image_id/image),
    zostawiamy ją bez zmian.
    """
    if df is None or len(df) == 0:
        return df

    df = df.copy()

    # źródło identyfikatora pliku
    if "file_name" in df.columns:
        key_col = "file_name"
    elif "image_id" in df.columns:
        key_col = "image_id"
    elif "image" in df.columns:
        key_col = "image"
    else:
        # brak kolumny identyfikującej plik – nic nie zmieniamy
        return df

    # standaryzacja file_name (bez ścieżek)
    df["file_name"] = df[key_col].astype(str).map(lambda x: Path(x).name)

    # upewniamy się, że są subject_en / subject_pl
    for col in ("subject_en", "subject_pl"):
        if col not in df.columns:
            df[col] = ""

    # kolejność kolumn: file_name, subject_en, subject_pl, reszta
    cols = ["file_name", "subject_en", "subject_pl"]
    other = [c for c in df.columns if c not in cols]
    df = df[cols + other]

    df["file_name"] = df["file_name"].astype(str)
    return df

def _cols_or_none(df: pd.DataFrame | None):
    return list(df.columns) if df is not None else None

# zakładamy, że df_lvis, df_clip, df_ocr są już zdefiniowane
df_lvis = _normalize_subject_df(df_lvis)
df_clip = _normalize_subject_df(df_clip)
df_ocr  = _normalize_subject_df(df_ocr)

print("[norm] df_lvis  :", _cols_or_none(df_lvis))
print("[norm] df_clip  :", _cols_or_none(df_clip))
print("[norm] df_ocr   :", _cols_or_none(df_ocr))

[norm] df_lvis  : ['file_name', 'subject_en', 'subject_pl']
[norm] df_clip  : ['file_name', 'subject_en', 'subject_pl']
[norm] df_ocr   : ['file_path', 'n_lines', 'total_chars', 'full_text', 'has_text']


### 8. Kandydaci subjectów (LVIS + CLIP)

In [11]:
"""
8. Kandydaci subjects per-obraz:
- LVIS: unikalne etykiety EN/PL dla danego pliku (max topk, z progiem score≥conf jeśli kolumna 'score' istnieje).
- CLIP: sceny EN/PL (jeśli są).
"""

LVIS_TOP_DEFAULT = 12

def _split_semis(s: str):
    return [t.strip() for t in str(s or "").split(";") if t and t.strip()]

def candidates_for(
    fname: str,
    conf: float = 0.45,
    topk: int = LVIS_TOP_DEFAULT,
):
    cand_pl, cand_en = [], []

    # LVIS – korzystamy z pliku lvis_subjects_en_pl.csv
    if not df_lvis.empty:
        d = df_lvis[df_lvis["file_name"] == fname]

        # jeśli mamy kolumnę score, filtruj po progu conf
        if "score" in d.columns:
            d = d[d["score"] >= conf]

        if not d.empty:
            en_list = (
                d["subject_en"]
                .dropna()
                .astype(str)
                .tolist()
            )
            pl_list = (
                d["subject_pl"]
                .dropna()
                .astype(str)
                .tolist()
            )

            # ograniczenie do topk unikalnych
            def _unique(seq):
                seen, out = set(), []
                for x in seq:
                    k = x.lower()
                    if k not in seen:
                        seen.add(k)
                        out.append(x)
                return out

            en_list = _unique(en_list)[: int(topk)]
            pl_list = _unique(pl_list)[: int(topk)]

            cand_en += en_list
            cand_pl += pl_list

    # CLIP – sceny EN/PL (jeśli df_clip istnieje)
    if "df_clip" in globals() and df_clip is not None and not df_clip.empty:
        d = df_clip[df_clip["file_name"] == fname]
        if not d.empty:
            cand_en += [
                str(x) for x in d["subject_en"].dropna().astype(str).tolist()
            ]
            cand_pl += [
                str(x) for x in d["subject_pl"].dropna().astype(str).tolist()
            ]

    # deduplikacja
    def dedup(seq):
        seen, out = set(), []
        for x in [s for s in seq if s]:
            k = x.lower()
            if k not in seen:
                seen.add(k)
                out.append(x)
        return out

    return dedup(cand_pl), dedup(cand_en)

### 9. Wybór i zatwierdzenie subjects

In [20]:
### 9. Interfejs wyboru subjectów (LVIS + CLIP + OCR → subject_human_selected.csv)

"""
Dla każdego file_name pokazuje:
- miniaturę obrazu (z P_PREV lub P_INPUTS),
- listę subjectów (PL/EN) z df_dc_subjects jako checkboksy,
- pole tekstowe na własne subjecty PL (oddzielone średnikami ';'),
- pole tekstowe na tekst z OCR (oddzielony średnikami ';'),
- checkbox + pole tekstowe na scenę (03_clip), zapisywaną w kolumnie 'universal'.

Po kliknięciu „Zapisz”:
- subjecty + ewentualne napisy z OCR trafiają do subject_human_selected.csv.
"""

from pathlib import Path
import pandas as pd
from IPython.display import display
import ipywidgets as W
from PIL import Image
import io, base64

# Bezpieczne wczytanie df_dc_subjects z pliku, jeśli nie istnieje w pamięci
if "df_dc_subjects" not in globals():
    P_DC_SUBJ = ROOT / "outputs" / "csv" / "dc_subjects_clip_lvis.csv"
    if not P_DC_SUBJ.exists():
        raise FileNotFoundError(f"Brak pliku z subjectami DC: {P_DC_SUBJ}")
    df_dc_subjects = pd.read_csv(P_DC_SUBJ)
    
P_SUBJ_SEL = ROOT / "outputs" / "csv" / "subject_human_selected.csv"
P_SUBJ_SEL.parent.mkdir(parents=True, exist_ok=True)

# wczytaj istniejący wybór, jeśli jest
if P_SUBJ_SEL.exists():
    df_sel = pd.read_csv(P_SUBJ_SEL)
else:
    df_sel = pd.DataFrame(columns=["file_name", "pl", "en", "universal", "selected"])

# lista plików do przeglądu
file_list = sorted(df_dc_subjects["file_name"].astype(str).unique().tolist())
idx = 0 if file_list else -1

w_prev = W.Button(description="◀︎ Poprzedni", layout=W.Layout(width="120px"))
w_next = W.Button(description="Następny ▶︎", layout=W.Layout(width="120px"))
w_save = W.Button(description="Zapisz", button_style="success", layout=W.Layout(width="120px"))
w_info = W.HTML()

w_img = W.HTML()   # miniatura
w_box = W.VBox()   # checkboksy

w_manual_pl = W.Textarea(
    placeholder="Dodatkowe subjecty po polsku, oddzielone średnikami ';'\nnp. pochód 1 maja; ćwikła; mleko w proszku",
    layout=W.Layout(width="100%", height="70px")
)

# tekst z OCR
w_ocr = W.Textarea(
    placeholder="Tekst z OCR (np. z transparentów), oddzielony średnikami ';'",
    layout=W.Layout(width="100%", height="160px")
)
w_ocr_chk = W.Checkbox(
    value=False,
    description="Dodać tekst z OCR jako subject (PL)",
    indent=False,
    layout=W.Layout(width="260px"),
)

# scena CLIP (universal)
w_scene_chk = W.Checkbox(
    value=False,
    description="Kolumna universal",
    indent=False,
    layout=W.Layout(width="200px")
)
w_scene_text = W.Text(
    placeholder="np. wnętrze sklepu",
    layout=W.Layout(width="100%")
)


def _row_selected(fname: str, pl: str, en: str, universal: str) -> bool:
    if df_sel.empty:
        return False
    m = (
        (df_sel["file_name"].astype(str) == fname)
        & (df_sel["pl"].astype(str) == pl)
        & (df_sel["en"].astype(str) == en)
        & (df_sel["universal"].astype(str) == universal)
        & (df_sel["selected"] == 1)
    )
    return bool(m.any())


def _get_saved_scene(fname: str):
    """Zwraca (universal, selected) dla zapisanej sceny, jeśli istnieje."""
    if df_sel.empty:
        return "", False
    m = (
        (df_sel["file_name"].astype(str) == fname)
        & (df_sel["pl"].astype(str) == "")
        & (df_sel["en"].astype(str) == "")
        & (df_sel["universal"].astype(str) != "")
    )
    if not m.any():
        return "", False
    row = df_sel[m].iloc[0]
    return str(row["universal"]), bool(row.get("selected", 1) == 1)


def _get_ocr_text(fname: str) -> str:
    """
    Zwraca tekst OCR dla danego pliku jako 'frag1; frag2; ...'
    na podstawie df_ocr (ocr_results.csv z kolumnami file_name, full_text).
    """
    if "df_ocr" not in globals() or df_ocr is None or df_ocr.empty:
        return ""
    if "file_name" not in df_ocr.columns or "full_text" not in df_ocr.columns:
        return ""
    d = df_ocr[df_ocr["file_name"].astype(str) == fname]
    if d.empty:
        return ""

    text = str(d["full_text"].iloc[0])
    # rozbicie na linie → frazy
    parts = [s.strip() for s in text.replace("\r", "\n").split("\n") if s.strip()]
    if not parts:
        return text.strip()

    # deduplikacja
    seen, out = set(), []
    for t in parts:
        if t not in seen:
            seen.add(t)
            out.append(t)
    return "; ".join(out)


def _load_preview(fname: str) -> Image.Image:
    cand = [P_PREV / fname, P_INPUTS / fname]
    for p in cand:
        if p.exists():
            return Image.open(p).convert("RGB")
    raise FileNotFoundError(fname)


def _pil_to_dataurl(im: Image.Image, maxw: int = 800) -> str:
    w, h = im.size
    if w > maxw:
        im = im.resize((maxw, int(h * maxw / w)))
    buf = io.BytesIO()
    im.save(buf, format="JPEG", quality=92)
    b64 = base64.b64encode(buf.getvalue()).decode("ascii")
    return f"data:image/jpeg;base64,{b64}"


def render_subjects():
    w_box.children = ()
    w_manual_pl.value = ""
    w_ocr.value = ""
    w_ocr_chk.value = False
    w_scene_chk.value = False
    w_scene_text.value = ""

    if idx < 0 or idx >= len(file_list):
        w_info.value = "<b>Brak plików.</b>"
        w_img.value = ""
        return

    fname = file_list[idx]
    rows = df_dc_subjects[df_dc_subjects["file_name"].astype(str) == fname].copy()

    # miniatura
    try:
        img = _load_preview(fname)
        dataurl = _pil_to_dataurl(img)
        w_img.value = f"<div style='max-width:800px'><img src='{dataurl}' style='width:100%'/></div>"
    except FileNotFoundError:
        w_img.value = "<i>Brak podglądu.</i>"

    # checkboksy subjectów
    checks = []
    for _, r in rows.iterrows():
        pl = str(r["pl"])
        en = str(r["en"])
        uni = str(r.get("universal", ""))

        label = pl if pl else en if en else uni
        if pl and en:
            label = f"{pl} / {en}"

        chk = W.Checkbox(
            value=_row_selected(fname, pl, en, uni),
            description=label,
            indent=False,
        )
        chk._meta = {
            "file_name": fname,
            "pl": pl,
            "en": en,
            "universal": uni,
        }
        checks.append(chk)

    if not checks:
        checks.append(W.HTML("<i>Brak subjectów dla tego pliku.</i>"))

    w_box.children = tuple(checks)

    # scena CLIP – najpierw PL, potem EN
    saved_uni, saved_sel = _get_saved_scene(fname)
    scene_suggest = ""

    if (
        not saved_uni
        and "df_clip" in globals()
        and df_clip is not None
        and not df_clip.empty
    ):
        d_clip = df_clip[df_clip["file_name"].astype(str) == fname]
        if not d_clip.empty:
            if "subject_pl" in d_clip.columns and pd.notna(d_clip["subject_pl"].iloc[0]):
                scene_suggest = str(d_clip["subject_pl"].iloc[0])
            elif "subject_en" in d_clip.columns and pd.notna(d_clip["subject_en"].iloc[0]):
                scene_suggest = str(d_clip["subject_en"].iloc[0])

    if saved_uni:
        w_scene_text.value = saved_uni
        w_scene_chk.value = saved_sel
    else:
        w_scene_text.value = scene_suggest
        w_scene_chk.value = False

    # tekst z OCR – propozycja z df_ocr dla danego file_name
    w_ocr.value = _get_ocr_text(fname)

    w_info.value = f"<b>{fname}</b> &nbsp;({idx+1} / {len(file_list)})"


def on_prev(_):
    global idx
    if idx > 0:
        idx -= 1
        render_subjects()


def on_next(_):
    global idx
    if idx < len(file_list) - 1:
        idx += 1
        render_subjects()


def on_save(_):
    global df_sel
    if idx < 0 or idx >= len(file_list):
        return

    fname = file_list[idx]
    rows = []

    # 1) wiersze z checkboxów (LVIS/CLIP subjects)
    for w in w_box.children:
        if not isinstance(w, W.Checkbox):
            continue
        meta = getattr(w, "_meta", None)
        if meta is None:
            continue
        row = {
            "file_name": meta["file_name"],
            "pl": meta["pl"],
            "en": meta["en"],
            "universal": meta["universal"],
            "selected": 1 if w.value else 0,
        }
        rows.append(row)

    # 2) dodatkowe subjecty PL wpisane ręcznie
    extra = [
        t.strip() for t in w_manual_pl.value.split(";")
        if t and t.strip()
    ]
    for pl in extra:
        rows.append({
            "file_name": fname,
            "pl": pl,
            "en": "",
            "universal": "",
            "selected": 1,
        })

    # 3) tekst z OCR → opcjonalnie jako subjecty PL
    ocr_text = w_ocr.value.strip()
    if w_ocr_chk.value and ocr_text:
        for pl in [t.strip() for t in ocr_text.split(";") if t.strip()]:
            rows.append({
                "file_name": fname,
                "pl": pl,
                "en": "",
                "universal": "",
                "selected": 1,
            })

    # 4) scena CLIP / ręczna → universal
    scene_text = w_scene_text.value.strip()
    if w_scene_chk.value and scene_text:
        rows.append({
            "file_name": fname,
            "pl": "",
            "en": "",
            "universal": scene_text,
            "selected": 1,
        })

    df_new = pd.DataFrame(rows)

    # usuń stare wpisy dla tego file_name i wstaw nowe
    df_sel = df_sel[df_sel["file_name"].astype(str) != fname]
    df_sel = pd.concat([df_sel, df_new], ignore_index=True)

    df_sel.to_csv(P_SUBJ_SEL, index=False)
    print(f"[DC] Zapisano wybory dla {fname} do:", P_SUBJ_SEL)

    on_next(None)


w_prev.on_click(on_prev)
w_next.on_click(on_next)
w_save.on_click(on_save)

controls = W.HBox([w_prev, w_next, w_save, w_info])
ui = W.VBox([
    controls,
    W.HBox([
        w_img,
        W.VBox([
            w_box,
            w_manual_pl,
            w_ocr,
            w_ocr_chk,
            W.HBox([w_scene_chk, w_scene_text]),
        ])
    ])
])

display(ui)
render_subjects()

### 10. Scalanie df_dc_subjects z subject_human_selected

In [19]:
### 7.Z Scalanie df_dc_subjects z subject_human_selected → human_selected

"""
Tworzy finalną tabelę DC z uwzględnieniem wyborów człowieka.

Zasady:
- Jeśli subject_human_selected.csv istnieje i zawiera selected==1,
  do eksportu trafiają tylko te wiersze.
- Jeśli plik nie istnieje lub nie ma żadnych selected==1,
  eksportujemy wszystkie wiersze z df_dc_subjects.

Wyjście:
    outputs/csv/dublin_core_subjects_final.csv
    kolumny: file_name, pl, en, universal
"""

from pathlib import Path
import pandas as pd

P_DC_SUBJ   = ROOT / "outputs" / "csv" / "dc_subjects_clip_lvis.csv"
P_SUBJ_SEL  = ROOT / "outputs" / "csv" / "subject_human_selected.csv"
P_DC_FINAL  = ROOT / "outputs" / "csv" / "dublin_core_subjects_final.csv"
P_DC_FINAL.parent.mkdir(parents=True, exist_ok=True)

# baza: wszystkie subjecty z LVIS+CLIP (fallback)
df_all = pd.read_csv(P_DC_SUBJ) if P_DC_SUBJ.exists() else df_dc_subjects.copy()

# wybory człowieka (mogą nie istnieć)
if P_SUBJ_SEL.exists():
    df_sel = pd.read_csv(P_SUBJ_SEL)
else:
    df_sel = pd.DataFrame(columns=["file_name", "pl", "en", "universal", "selected"])

if (
    not df_sel.empty
    and "selected" in df_sel.columns
    and df_sel["selected"].sum() > 0
):
    # używamy WYŁĄCZNIE wierszy wybranych przez człowieka
    df_final = (
        df_sel[df_sel["selected"] == 1]
        .loc[:, ["file_name", "pl", "en", "universal"]]
        .copy()
    )
else:
    # brak zaznaczeń – bierzemy wszystko z automatu
    df_final = df_all.loc[:, ["file_name", "pl", "en", "universal"]].copy()

df_final.to_csv(P_DC_FINAL, index=False)
print("[DC] Zapisano finalny eksport DC do:", P_DC_FINAL)
print("[DC] df_final shape:", df_final.shape)
df_final.head(20)

[DC] Zapisano finalny eksport DC do: /Users/olga/MetaLogic/outputs/csv/dublin_core_subjects_final.csv
[DC] df_final shape: (31, 4)


,file_name,pl,en,universal
8,0077.jpg,szyld,signboard,NaN
10,0077.jpg,słup telefoniczny,telephone_pole,NaN
11,0077.jpg,tłum na ulicy,crowd of people on the street,NaN
12,0077.jpg,kalinowka,NaN,NaN
14,0004.jpg,butelka wina,wine_bottle,NaN
15,0004.jpg,etykieta,tag,NaN
19,0004.jpg,ćwikła,NaN,NaN
20,0004.jpg,dżem,NaN,NaN
21,0004.jpg,UWAGA!,NaN,NaN
22,0004.jpg,Sprzedaż mleka,NaN,NaN
